# Doawnloads and Stores IPO S-1 Filings as .txt Files

* Uses EDGAR search
 * https://www.sec.gov/edgar/searchedgar/companysearch.html
* Uses NASDAQ IPO lists
 * https://www.nasdaq.com/markets/ipos/
* Skips ambiguous company names (do manually)
* Takes latest file if multiple files are available (S-1, S-1/A etc.)

In [1]:
import edgar
import nasdaq

from pathlib import Path

### Scrape NASDAQ IPO Lists

In [2]:
df_symbols = nasdaq.get_ipo_list('2000-01-01')

date range: odict_keys(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06', '2000-07', '2000-08', '2000-09', '2000-10', '2000-11', '2000-12', '2001-01', '2001-02', '2001-03', '2001-04', '2001-05', '2001-06', '2001-07', '2001-08', '2001-09', '2001-10', '2001-11', '2001-12', '2002-01', '2002-02', '2002-03', '2002-04', '2002-05', '2002-06', '2002-07', '2002-08', '2002-09', '2002-10', '2002-11', '2002-12', '2003-01', '2003-02', '2003-03', '2003-04', '2003-05', '2003-06', '2003-07', '2003-08', '2003-09', '2003-10', '2003-11', '2003-12', '2004-01', '2004-02', '2004-03', '2004-04', '2004-05', '2004-06', '2004-07', '2004-08', '2004-09', '2004-10', '2004-11', '2004-12', '2005-01', '2005-02', '2005-03', '2005-04', '2005-05', '2005-06', '2005-07', '2005-08', '2005-09', '2005-10', '2005-11', '2005-12', '2006-01', '2006-02', '2006-03', '2006-04', '2006-05', '2006-06', '2006-07', '2006-08', '2006-09', '2006-10', '2006-11', '2006-12', '2007-01', '2007-02', '2007-03', '2007-04', '2007-05

C:\Users\Void\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
C:\Users\Void\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:87: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [ ]:
df_symbols.head(3)

,Company Name,Date Priced,Market,Offer Amount,Price,Shares,Symbol,0,1,2
Symbol,,,,,,,,,,
APL,TARGA PIPELINE PARTNERS LP,2000-01-28,American Stock Exchange,19500000.0,13.0,1.5e+06,APL,NaN,NaN,NaN
ASPM,ASPECT MEDICAL SYSTEMS INC,2000-01-28,NASDAQ,52500000.0,15.0,3.5e+06,ASPM,NaN,NaN,NaN
CAMZ,CAMINUS CORP,2000-01-28,NASDAQ,69952000.0,16.0,4.372e+06,CAMZ,NaN,NaN,NaN


### Batch Download

In [ ]:
#batch download
counter = 0

for x in df_symbols['Symbol']:
    try:
        counter += 1
        print('\n( ' + str(counter) + ' / ' + str(df_symbols.shape[0]) + ' ) ' + str(x))

        #check if exists
        if Path("./Data/" + x + ".htm").is_file():
            print(x + ' data already exists, skipping...')
            continue
        # file exists

        #create company
        tmpEdgar = edgar.Edgar()
        possible_companies = tmpEdgar.findCompanyName(df_symbols.loc[x]['Company Name'])
        print('possible_companies:', possible_companies)

        #validate
        if len(possible_companies) == 0:
            print('no possible companies:', x)
            continue

        name = possible_companies[0]
        cik = tmpEdgar.getCikByCompanyName(possible_companies[0])
        company = edgar.Company(name, cik)

        #look for S-1
        tree = company.getAllFilings(filingType = "S-1")
        docs = edgar.getDocuments(tree, noOfDocuments=5)

        #if multiple amendments exist
        #filter ones with length lower than 500K (statistically the mean is higher)
        docs = [d for d in docs if len(d) > 500000]

        #write file
        if len(docs) > 0:
            with open("./Data/" + x + ".htm", "w", encoding="utf-8") as f:
                f.write(docs[0].decode("utf-8"))
                print('Got data for ' + x + ' chose ' + str(len(docs[0])))
    except Exception as e:
        print(x, e)


( 1 / 3665 ) APL
possible_companies: ['TARGA PIPELINE PARTNERS LP']
doc 348472 https://www.sec.gov/Archives/edgar/data/1092914/000119312509203562/ds1a.htm
doc 339088 https://www.sec.gov/Archives/edgar/data/1092914/000119312509187003/ds1.htm
doc 335798 https://www.sec.gov/Archives/edgar/data/1092914/000119312509132490/ds1.htm
doc 11722 https://www.sec.gov/Archives/edgar/data/1092914/000095011600000079/
doc 10662 https://www.sec.gov/Archives/edgar/data/1092914/000095011600000010/

( 2 / 3665 ) ASPM
possible_companies: ['ASPECT MEDICAL SYSTEMS INC']
doc 10302 https://www.sec.gov/Archives/edgar/data/886235/000095013500000274/
doc 11183 https://www.sec.gov/Archives/edgar/data/886235/000095013500000272/
doc 10477 https://www.sec.gov/Archives/edgar/data/886235/000095013500000258/
doc 12064 https://www.sec.gov/Archives/edgar/data/886235/000095013500000197/
doc 10830 https://www.sec.gov/Archives/edgar/data/886235/000095013500000061/

( 3 / 3665 ) CAMZ
possible_companies: ['CAMINUS CORP']

( 4 

doc 10484 https://www.sec.gov/Archives/edgar/data/1091953/000095013000000725/
doc 11014 https://www.sec.gov/Archives/edgar/data/1091953/000095013000000499/
doc 10836 https://www.sec.gov/Archives/edgar/data/1091953/000095013000000243/
doc 11546 https://www.sec.gov/Archives/edgar/data/1091953/000095013000000006/
doc 14378 https://www.sec.gov/Archives/edgar/data/1091953/000095013099006664/

( 32 / 3665 ) APRS
possible_companies: ['APROPOS TECHNOLOGY INC']
doc 10309 https://www.sec.gov/Archives/edgar/data/1098803/000095013100001295/
doc 10309 https://www.sec.gov/Archives/edgar/data/1098803/000095013100001262/
doc 11371 https://www.sec.gov/Archives/edgar/data/1098803/000095013100000994/
doc 10662 https://www.sec.gov/Archives/edgar/data/1098803/000095013100000480/
doc 11900 https://www.sec.gov/Archives/edgar/data/1098803/000095013100000357/

( 33 / 3665 ) JCWW
possible_companies: ['LIFEQUEST WORLD CORP']
doc 10308 https://www.sec.gov/Archives/edgar/data/1060888/000089710199000601/
doc 33958 

possible_companies: ['FN ESTATE INC']
doc 10659 https://www.sec.gov/Archives/edgar/data/1092536/000091205700004301/
doc 15262 https://www.sec.gov/Archives/edgar/data/1092536/000091205700003587/
doc 10661 https://www.sec.gov/Archives/edgar/data/1092536/000091205700001455/
doc 10484 https://www.sec.gov/Archives/edgar/data/1092536/000091205799011119/
doc 10661 https://www.sec.gov/Archives/edgar/data/1092536/000091205799009036/

( 60 / 3665 ) BUYX
possible_companies: ['BIDORBUY.COM INC', 'BUY COM INC', 'BUY IT CHEAP COM INC /DE', 'BUYERSONLINECOM INC', 'BUYERZONE COM INC', 'BUYIT COM INC', 'EATON VANCE TAX-MANAGED BUY-WRITE INCOME FUND', 'EATON VANCE TAX-MANAGED BUYWRITE INCOME FUND', 'HOTBUYER COM INC', 'IBUYDIGITAL.COM, INC.', 'MEDIBUY COM INC', 'NUVEEN S&P 500 BUYWRITE INCOME FUND', 'SHOEBUY COM INC', 'TELCOBUY COM INC', 'UBUYJAWS COM INC', 'WILSON BUY-WRITE INCOME FUND, LP']

( 61 / 3665 ) AMWW
possible_companies: ['AIMS WORLDWIDE INC']

( 62 / 3665 ) AGEN
possible_companies: ['AGENUS 

doc 11367 https://www.sec.gov/Archives/edgar/data/1100983/000091205700002846/
doc 13839 https://www.sec.gov/Archives/edgar/data/1100983/000091205799009843/

( 84 / 3665 ) TLCT
possible_companies: ['DIRECTV BROADBAND INC']
doc 10488 https://www.sec.gov/Archives/edgar/data/1102448/000089161800001777/
doc 10836 https://www.sec.gov/Archives/edgar/data/1102448/000089161800001737/
doc 11553 https://www.sec.gov/Archives/edgar/data/1102448/000089161800001649/
doc 11015 https://www.sec.gov/Archives/edgar/data/1102448/000089161800001586/
doc 12248 https://www.sec.gov/Archives/edgar/data/1102448/000089161800001319/

( 85 / 3665 ) SEA
possible_companies: ['NEW DRAGON ASIA CORP']

( 86 / 3665 ) WBSN
possible_companies: ['WEBSENSE INC']
doc 10310 https://www.sec.gov/Archives/edgar/data/1098277/000093639200000148/
doc 10663 https://www.sec.gov/Archives/edgar/data/1098277/000093639200000146/
doc 13673 https://www.sec.gov/Archives/edgar/data/1098277/000109581100000445/
doc 12079 https://www.sec.gov/Arc

doc 12956 https://www.sec.gov/Archives/edgar/data/1101179/000095011600000191/

( 109 / 3665 ) IMPV
IMPV data already exists, skipping...

( 110 / 3665 ) ACTI
possible_companies: ['ACTIVCARD SA']

( 111 / 3665 ) ASIQ
possible_companies: ['AS-IP TECH INC']

( 112 / 3665 ) LOUD
possible_companies: ['LOUDEYE CORP']

( 113 / 3665 ) SHOC
possible_companies: ['LENCO MOBILE INC.']

( 114 / 3665 ) RVSN
possible_companies: ['RADVISION LTD']

( 115 / 3665 ) FAIM
possible_companies: []
no possible companies: FAIM

( 116 / 3665 ) DTAS
possible_companies: ['DIGITAS INC']
doc 10308 https://www.sec.gov/Archives/edgar/data/1100885/000092701600000815/
doc 10310 https://www.sec.gov/Archives/edgar/data/1100885/000092701600000800/
doc 10487 https://www.sec.gov/Archives/edgar/data/1100885/000092701600000697/
doc 11190 https://www.sec.gov/Archives/edgar/data/1100885/000092701600000664/
doc 16850 https://www.sec.gov/Archives/edgar/data/1100885/000092701600000448/

( 117 / 3665 ) HOMG
possible_companies: ['HOM

...

doc 1808801 https://www.sec.gov/Archives/edgar/data/1133062/000114420409038655/v155454_s1a.htm
doc 1901547 https://www.sec.gov/Archives/edgar/data/1133062/000114420409014885/v143331_s1.htm
Got data for JLWT chose 935479

( 675 / 3665 ) DTSI
possible_companies: ['DTS, INC.', 'ONCO TREE DTS, INC.']
doc 1646514 https://www.sec.gov/Archives/edgar/data/1226308/000095014803002620/v94004a1sv1za.htm
doc 1645251 https://www.sec.gov/Archives/edgar/data/1226308/000095014803002578/v94004orsv1.htm
doc 22593 https://www.sec.gov/Archives/edgar/data/1226308/000095014803001683/v88961mesv1mef.htm
doc 1612748 https://www.sec.gov/Archives/edgar/data/1226308/000095014803001648/v88961a3sv1za.htm
doc 1603849 https://www.sec.gov/Archives/edgar/data/1226308/000095014803001562/v88961a2sv1za.htm
Got data for DTSI chose 1646514

( 676 / 3665 ) MOH
possible_companies: ['MOLINA HEALTHCARE INC']
doc 1464669 https://www.sec.gov/Archives/edgar/data/1179929/000119312504043377/ds1a.htm
doc 1466079 https://www.sec.gov/Ar

doc 1370702 https://www.sec.gov/Archives/edgar/data/1236038/000089161803004843/f89225a5sv1za.htm
doc 1368369 https://www.sec.gov/Archives/edgar/data/1236038/000089161803004700/f89225a4sv1za.htm
Got data for REDE chose 1365824

( 694 / 3665 ) AMIS
possible_companies: ['AMIS HOLDINGS INC']
doc 35030 https://www.sec.gov/Archives/edgar/data/1161963/000089161803004889/f92241mesv1mef.htm
doc 79723 https://www.sec.gov/Archives/edgar/data/1161963/000089161803004868/f92241a5sv1za.htm
doc 80421 https://www.sec.gov/Archives/edgar/data/1161963/000089161803004864/f92241a4sv1za.htm
doc 2825231 https://www.sec.gov/Archives/edgar/data/1161963/000089161803004826/f92241a3sv1za.htm
doc 2827049 https://www.sec.gov/Archives/edgar/data/1161963/000089161803004674/f92241a2sv1za.htm
Got data for AMIS chose 2825231

( 695 / 3665 ) JRN
possible_companies: ['JOURNAL COMMUNICATIONS INC']
doc 2800248 https://www.sec.gov/Archives/edgar/data/1232241/000119312504095319/ds1a.htm
doc 2731993 https://www.sec.gov/Archives

doc 2101519 https://www.sec.gov/Archives/edgar/data/1160241/000104746903030598/a2118589zs-1a.htm
doc 2071331 https://www.sec.gov/Archives/edgar/data/1160241/000104746903028762/a2117457zs-1a.htm
Got data for DNET chose 2116365

( 714 / 3665 ) ACUS
possible_companies: ['ACUSPHERE INC']
doc 1319620 https://www.sec.gov/Archives/edgar/data/1115143/000104746904030479/a2144180zs-1a.htm
doc 1323438 https://www.sec.gov/Archives/edgar/data/1115143/000104746904028844/a2143097zs-1.htm
doc 1881221 https://www.sec.gov/Archives/edgar/data/1115143/000095013503005073/b47005a4sv1za.htm
doc 1896545 https://www.sec.gov/Archives/edgar/data/1115143/000095013503004835/b47005a3sv1za.htm
doc 1881842 https://www.sec.gov/Archives/edgar/data/1115143/000095013503004545/b47005a2sv1za.htm
Got data for ACUS chose 1319620

( 715 / 3665 ) LKQX
possible_companies: ['LKQ ALL MODELS CORP.', 'LKQ BEST AUTOMOTIVE CORP.', 'LKQ CORP', 'LKQ GORHAM AUTO PARTS CORP.', 'LKQ GREAT LAKES CORP.', 'LKQ HUNTS POINT AUTO PARTS CORP.', 

doc 876357 https://www.sec.gov/Archives/edgar/data/927829/000104746903033876/a2120447zs-1a.htm
Got data for NTMD chose 847892

( 731 / 3665 ) CFHI
possible_companies: ['COAST FINANCIAL HOLDINGS INC']
doc 2536817 https://www.sec.gov/Archives/edgar/data/1262276/000119312505186211/ds1a.htm
doc 2519778 https://www.sec.gov/Archives/edgar/data/1262276/000119312505167055/ds1.htm
Got data for CFHI chose 2536817

( 732 / 3665 ) KNBT
possible_companies: ['KNBT BANCORP INC']
doc 7604282 https://www.sec.gov/Archives/edgar/data/1236964/000119312503033709/ds1a.htm
doc 7526167 https://www.sec.gov/Archives/edgar/data/1236964/000119312503029161/ds1a.htm
doc 7477319 https://www.sec.gov/Archives/edgar/data/1236964/000102140803009329/ds1a.htm
doc 8100517 https://www.sec.gov/Archives/edgar/data/1236964/000092838503001774/ds1.htm
Got data for KNBT chose 7604282

( 733 / 3665 ) KNTA
possible_companies: ['KINTERA INC']
doc 937873 https://www.sec.gov/Archives/edgar/data/1117119/000104746904033028/a2144497zs-1a

doc 735416 https://www.sec.gov/Archives/edgar/data/1231868/000095015203007226/l91798asv1.txt
Got data for ISG chose 777787

( 749 / 3665 ) CENF
